# Восстановленный ноутбук (как в логах/kernel dump)

## Import библиотек и модулей


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os, sys
import json

from pathlib import Path
import re
from datetime import datetime, timedelta
# --- Настройка путей и sys.path ---
# Добавляем корневую директорию проекта в sys.path для импорта кастомных модулей
PROJECT_ROOT = Path().cwd().parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))
from src.config import config
from src.logger import logger
from src.database import clickhouse_engine, postgres_engine, ipdr_engine   
import pandas as pd, pyarrow
import numpy as np
import matplotlib.pyplot as plt
import os, sys
from pathlib import Path
import re
from datetime import datetime, timedelta
# --- Настройка путей и sys.path ---
# Добавляем корневую директорию проекта в sys.path для импорта кастомных модулей
PROJECT_ROOT = Path().cwd().parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))
from src.config import config
from src.logger import logger
from src.database import clickhouse_engine

Configuration loaded successfully.


2025-11-13 08:28:58,972 | my_logger - INFO - ✅ PostgreSQL engine создан | /data/aturov/scoring/src/database.py:21
2025-11-13 08:28:59,012 | my_logger - INFO - ✅ ClickHouse engine создан | /data/aturov/scoring/src/database.py:36
2025-11-13 08:28:59,013 | my_logger - INFO - ✅ IPDR ClickHouse engine создан | /data/aturov/scoring/src/database.py:46


## Constant variables and paths

In [2]:
CURRENT_DATE = datetime.now().strftime("%Y-%m-%d")
TARGET_COL = 'target'
# Key params (tune as needed)
COUNT_WEEKS = 13                # implies history length N = 12 COUNT_WEEKS = [5, 13]
OVERDUE_DAYS_MAX = 30           # example threshold for 'bad' target OVERDUE_DAYS_MAX = [10, 30]
TOTAL_OVERDUE = 90               # example threshold for total overdue days/amount TOTAL_OVERDUE = [60, 90]
DATE_END = "2025-04-30"         # cap for date_open (filtering); set as needed

NAME_DATAFRAME_WEEKS = f'features_weeks_{COUNT_WEEKS - 1}_{OVERDUE_DAYS_MAX}_{TOTAL_OVERDUE}' # имя файла с признаками по неделям for LSTMs models
NAME_DATAFRAME_TABLE = f'features_table_{COUNT_WEEKS - 1}_{OVERDUE_DAYS_MAX}_{TOTAL_OVERDUE}' # имя файла с табличными признаками
NAME_DATAFRAME_BANKING = f'features_table_banking_{COUNT_WEEKS - 1}_{OVERDUE_DAYS_MAX}_{TOTAL_OVERDUE}' # имя файла с табличными признаками банковскими признаками

## Dowlnload данных из ClickHouse и Postgres

In [3]:
def make_query(engine, number_weeks):
    """
    Функция для выполнения SQL-запроса к базе данных и получения данных в виде DataFrame.
    Параметры:
    - engine: SQLAlchemy engine для подключения к базе данных.
    Возвращает:
    - DataFrame с результатами запроса.
    """
    query = f"""
        SELECT
            w.*,
            ce.id_request,
            ce.phone_eldik, 
            ce.inn_eldik, 
            ce.id_credit, 
            ce.date_open, 
            ce.subscription_id, 
            ce.match_phone, 
            ce.match_inn, 
            ce.overdue_max, 
            ce.total_overdue,
            ce.status
        FROM (
            SELECT *,
                toDate(DT) AS dt_day
            FROM DWH.dm_datamart_weekly 
            --WHERE STATUS not in ('Idle') -- исключаем статус Idle у него тот же cust_id 
        ) AS w
        GLOBAL INNER JOIN (
            SELECT 
                ce.id_request,
                ce.phone_eldik, 
                ce.inn_eldik, 
                ce.id_credit, 
                ce.date_open, 
                toUInt64(ce.subscription_id) AS subscription_id,
                ce.match_phone, 
                ce.match_inn, 
                IFNULL(ce.overdue_max, 0) AS overdue_max,
                IFNULL(ce.total_overdue, 0) AS total_overdue,
                ce.status
            FROM data_science.credits_subs_eldik_clean AS ce
        ) AS ce
        ON w.SUBS_ID = ce.subscription_id  and w.DT = toStartOfWeek(ce.date_open - INTERVAL {number_weeks} WEEK - INTERVAL 1 DAY, 1) 
        -- DT - дата начала интервала в понедельник, в понедельник данные загружаются за предыдущую неделю и не всегда успевают попасть в витрину
        WHERE 1=1
        --and w.SUBS_ID =  1993788
        
    """    
    data = pd.read_sql(query, engine)
    return data

In [4]:
#df_features_parts = []
#for number_week in range(1, COUNT_WEEKS):  # от 1 до 12 недель включительно
#    print(f"COUNT_WEEKS = {number_week}")
#    df_part = make_query(clickhouse_engine, number_weeks=number_week)
#    if df_part is None or df_part.empty:
#        logger.warning(f"COUNT_WEEKS = {number_week}, пустой датафрейм, пропускаем")
#        continue
#    df_part['count_weeks'] = number_week
#    logger.info(f"COUNT_WEEKS = {number_week}, shape = {df_part.shape}")
#    df_features_parts.append(df_part)
#
#df_features = pd.concat(df_features_parts, ignore_index=True) if df_features_parts else pd.DataFrame()
#
## приведение типов и сортировка
#if 'DT' in df_features.columns:
#    df_features['DT'] = pd.to_datetime(df_features['DT'], errors='coerce')
#df_features.sort_values(['subscription_id', 'id_credit', 'DT'], ascending=[True, True, False], inplace=True)
#df_features = df_features.query('count_weeks != 0')  # удаляем нулевую неделю (пересечение с датой открытия кредита)
#df_features.to_parquet(config.environment.data_raw_path / f'aturov_features_{CURRENT_DATE}.parquet', index=False)

In [5]:
df_features = pd.read_parquet(config.environment.data_raw_path / 'aturov_features_2025-11-12.parquet')
logger.info(f"Raw features shape: {df_features.shape}")
#df_features = df_features.query(f'count_weeks <= {COUNT_WEEKS - 1}') # оставляем только нужное количество недель
#logger.info(f"Initial features shape: {df_features.shape}")

# formating
df_features['TRANZ_FLAG'] = df_features['TRANZ_FLAG'].astype('boolean')
df_features['FLAG_4G'] = df_features['FLAG_4G'].astype('boolean')
df_features['ACTIVE_IND'] = df_features['ACTIVE_IND'].astype(str)
df_features['FLAG_ABONKA'] = df_features['FLAG_ABONKA'].astype('boolean')
df_features['MY_BEELINE_USER'] = df_features['MY_BEELINE_USER'].astype('boolean')
df_features['BALANCE_USER'] = df_features['BALANCE_USER'].astype('boolean')
df_features['MULTIPLAY'] = df_features['MULTIPLAY'].astype('boolean')
df_features['M2M_FLAG'] = df_features['M2M_FLAG'].astype('boolean')
df_features['match_inn'] = df_features['match_inn'].astype('boolean')
df_features['match_phone'] = df_features['match_phone'].astype('boolean')
# преобразование дат и создание новых признаков на основе разницы дат
df_features['date_open'] = pd.to_datetime(df_features['date_open'], errors='coerce')
df_features.dropna(subset=['date_open'], inplace=True)  # удаляем строки с некорректной датой открытия
#df_features = df_features[df_features['date_open'] <= pd.to_datetime(DATE_END)]  # фильтрация по дате открытия
#logger.info(f"Filtered by date_open <= {DATE_END}, shape: {df_features.shape}")
df_features['DATE_INACTIVE'] = pd.to_datetime(df_features['DATE_INACTIVE'], errors='coerce')
df_features['DATE_LAD'] = pd.to_datetime(df_features['DATE_LAD'], errors='coerce')
df_features['PRICE_CHANGE_DATE'] = pd.to_datetime(df_features['PRICE_CHANGE_DATE'], errors='coerce')
df_features['ACT_DATE'] = pd.to_datetime(df_features['ACT_DATE'], errors='coerce')
df_features['DATE_ABONKA'] = pd.to_datetime(df_features['DATE_ABONKA'], errors='coerce')
df_features['DATE_CONTRACT'] = pd.to_datetime(df_features['DATE_CONTRACT'], errors='coerce')
df_features['DT'] = pd.to_datetime(df_features['DT'], errors='coerce')
df_features['DT_END'] = (df_features['DT'] + pd.to_timedelta(7, unit='d')).dt.date # дата начала в понедельник влючительно и дата окончания в понедельник не включительно
df_features['DT_END'] = pd.to_datetime(df_features['DT_END'], errors='coerce')

# сразу посчитаем разницу дат 
df_features['days_from_dt_end_to_date_inactive'] = (df_features['DT_END'] - df_features['DATE_INACTIVE']).dt.days
df_features['days_from_dt_end_to_date_inactive'].fillna(-1, inplace=True)  # заполняем пропуски большим числом
df_features['days_from_dt_end_to_date_lad'] = (df_features['DT_END'] - df_features['DATE_LAD']).dt.days
df_features['days_from_dt_end_to_date_lad'].fillna(-1, inplace=True)  # заполняем пропуски большим числом
df_features['days_from_dt_end_to_price_change_date'] = (df_features['DT_END'] - df_features['PRICE_CHANGE_DATE']).dt.days
df_features['days_from_dt_end_to_price_change_date'].fillna(-1, inplace=True)  # заполняем пропуски большим числом
df_features['days_from_dt_end_to_act_date'] = (df_features['DT_END'] - df_features['ACT_DATE']).dt.days
df_features['days_from_dt_end_to_act_date'].fillna(-1, inplace=True)  # заполняем пропуски большим числом
df_features['days_from_dt_end_to_date_abonka'] = (df_features['DT_END'] - df_features['DATE_ABONKA']).dt.days
df_features['days_from_dt_end_to_date_abonka'].fillna(-1, inplace=True)  # заполняем пропуски большим числом
df_features['days_from_dt_end_to_date_contract'] = (df_features['DT_END'] - df_features['DATE_CONTRACT']).dt.days
df_features['days_from_dt_end_to_date_contract'].fillna(-1, inplace=True)  # заполняем пропуски большим числом
df_features.drop(columns=['DATE_INACTIVE', 'DATE_LAD', 'PRICE_CHANGE_DATE', 'ACT_DATE', 'DATE_ABONKA', 'DATE_CONTRACT', 'DT', 'DT_END'], inplace=True)
# удалим те у кого много параметров
#df_features.drop(columns=['CELL_MAX', 'CELL_ID', 'TAC', 'DEV_NAME'], inplace=True)
df_features[['CELL_ID', 'CELL_MAX', 'DEV_NAME', 'TAC', 'PRICE_PLAN_RU']] = df_features[['CELL_ID', 'CELL_MAX', 'DEV_NAME', 'TAC', 'PRICE_PLAN_RU']].astype(str).fillna("missing")


# Создаем таргет: плохой клиент = 1, хороший = 0
df_features['target'] = ((df_features['overdue_max'] >= OVERDUE_DAYS_MAX) | 
                        (df_features['total_overdue'] >= TOTAL_OVERDUE)| 
                        (df_features['status'].isin(['Отказано']))).astype(int)

df_features.drop(columns=['IMEI', 'CTN','SUBS_ID', 'dt_day', 'FIRST_SIM', 'overdue_max', 'total_overdue', 'status'
                          ,'USAGE_NUM_OFFNET_PAK', 'REVENUE_OFFNET_PAK'  # удалены из-за одного значения
                          ], inplace=True)
index_columns = ['id_request', 'subscription_id', 'phone_eldik', 
                 'inn_eldik', 'id_credit', 'date_open',
                 ]
sequential_columns = ['count_weeks']
#df_features.set_index(index_columns, inplace=True)
df_features.shape

2025-11-13 08:28:59,547 | my_logger - INFO - Raw features shape: (248062, 110) | /tmp/ipykernel_1310143/2054622685.py:2
/tmp/ipykernel_1310143/2054622685.py:34: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_features['days_from_dt_end_to_date_inactive'].fillna(-1, inplace=True)  # заполняем пропуски большим числом
/tmp/ipykernel_1310143/2054622685.py:36: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. Thi

(248062, 100)

In [6]:
df_features.columns

Index(['CUST_LEVEL', 'STATUS', 'REGION_CELL', 'PRICE_PLAN', 'PRICE_PLAN_RU',
       'PERIODICITY', 'SUBSCRIPTION_FEE', 'PREV_PRICE_PLAN', 'ORIG_PRICE_PLAN',
       'BALANCE_END', 'REVENUE_ABONKA', 'USAGE_ABONKA_TP', 'TRANZ_FLAG',
       'DAYS_WITHOUT_PAYMENT', 'TOTAL_RECHARGE', 'COUNT_RECHARGE', 'FLAG_4G',
       'USAGE_NUM_OUT', 'USAGE_OUT_ONNET_VOICE', 'USAGE_OUT_OFFNET_VOICE',
       'USAGE_OUT_CITY_VOICE', 'USAGE_OUT_INT_VOICE',
       'USAGE_OUT_INT_VOICE_RUSSIA', 'USAGE_IN_ONNET_VOICE',
       'USAGE_IN_OFFNET_VOICE', 'USAGE_VALUELESS_INTERNET', 'USAGE_INTERNET',
       'USAGE_INTERNET_2G', 'USAGE_INTERNET_3G', 'USAGE_INTERNET_LTE',
       'USAGE_INTERNET_3G_FREE', 'USAGE_INTERNET_LTE_FREE',
       'USAGE_OUT_OFFNET_O_VOICE', 'USAGE_OUT_OFFNET_MEGACOM_VOICE',
       'USAGE_IN_OFFNET_O_VOICE', 'USAGE_IN_OFFNET_MEGACOM_VOICE', 'COUNT_SMS',
       'REVENUE_VOICE', 'REVENUE_VOICE_TO_SERVICE', 'REVENUE_OUT_ONNET_VOICE',
       'REVENUE_OUT_OFFNET_VOICE', 'REVENUE_OUT_CITY_VOICE',
    

In [7]:
describe_features = pd.read_excel('/data/aturov/scoring/docs/Переменные_витрины_new_(3).xlsx')
describe_features.columns = ['features', 'description']
describe_features['features'] = describe_features['features'].str.lower()
describe_features

,features,description
0,dt,Дата начала расчётного периода
1,ctn,CTN (номер) абонента
2,subs_id,Ключ абонента
3,cust_level,"Тип клиента (B2B, B2C, Employee, etc)"
4,status,"Статус абонента (активный, блокированный, прио..."
...,...,...
93,multiplay,Входит в мультиплеи (юзер одного из приложений...
94,lifetime_total,Срок жизни абонента в нашей сети (дни)
95,m2m_flag,Флаг M2M
96,gender,Пол абонента по модели дата сайнтистов


In [8]:
target_stats = df_features.copy().query('count_weeks == 12')
target_stats = target_stats['target'].value_counts().sort_index()
target_stats

target
0    12785
1     7328
Name: count, dtype: int64

In [9]:
index_columns

['id_request',
 'subscription_id',
 'phone_eldik',
 'inn_eldik',
 'id_credit',
 'date_open']

In [10]:
#only_12_weeks = df_features.groupby(index_columns)['count_weeks'].count().sort_values().reset_index()
#
#only_12_weeks = only_12_weeks[only_12_weeks['count_weeks'] == COUNT_WEEKS - 1]
#only_12_weeks.set_index(index_columns, inplace=True)
#only_12_weeks.drop(columns=['count_weeks'], inplace=True)
#only_12_weeks.shape

In [11]:
df_features.set_index(index_columns, inplace=True)
#df_features = df_features.join(only_12_weeks, how='inner', on=index_columns)
#df_features.shape


In [12]:
# === Dtypes & index columns ===
datetime_cols = df_features.select_dtypes(include=['datetime64[ns]', 'datetimetz']).columns.tolist()
categorical_cols = df_features.select_dtypes(include=['object','category','boolean']).columns.tolist()
numeric_cols = df_features.select_dtypes(include=['float64','int64','float32','int32']).columns.tolist()


### Сохраним для дальенйшего обучения weeks

In [13]:

df_features.to_parquet(config.environment.data_processed_path / f'{NAME_DATAFRAME_WEEKS}_{CURRENT_DATE}.parquet', index=True)
logger.info(f"Features saved to {config.environment.data_processed_path / f'{NAME_DATAFRAME_WEEKS}_{CURRENT_DATE}.parquet'}")
# Сохранение категориальных признаков и числовых в json

with open(config.environment.data_processed_path / f'{NAME_DATAFRAME_WEEKS}_num_columns_{CURRENT_DATE}.json', 'w') as f:
    json.dump(numeric_cols, f)
with open(config.environment.data_processed_path / f'{NAME_DATAFRAME_WEEKS}_cat_columns_{CURRENT_DATE}.json', 'w') as f:
    json.dump(categorical_cols, f)


2025-11-13 08:29:02,479 | my_logger - INFO - Features saved to /data/aturov/scoring/data/processed/features_weeks_12_30_90_2025-11-13.parquet | /tmp/ipykernel_1310143/1195538844.py:2


In [14]:
df_features.shape

(248062, 94)

### Переведем недели в фичи для обычных моделей

In [15]:
numeric_cols = [col for col in numeric_cols if col not in (sequential_columns[0], TARGET_COL)]
categorical_cols = [col for col in categorical_cols if col not in (sequential_columns[0], TARGET_COL)]

tgt = df_features.reset_index(drop=False).drop_duplicates(subset=index_columns)[index_columns + [TARGET_COL]]
tgt = tgt.set_index(index_columns)
tgt.shape

(20852, 1)

In [16]:
def pivot_weeks(df: pd.DataFrame, id_cols, seq_col, numeric_cols, categorical_cols, COUNT_WEEKS: int):
    """Возвращает num_wide и cat_wide (индекс = id_cols)."""
    weeks = list(range(1, COUNT_WEEKS))
    df_reset = df.reset_index(drop=False)

    pnum = df_reset.pivot_table(index=id_cols, columns=seq_col, values=numeric_cols, aggfunc='first')
    if isinstance(pnum.columns, pd.MultiIndex):
        pnum.columns = [f"{feat}_w{int(wk) if str(wk).replace('.0','').isdigit() else wk}" for feat, wk in pnum.columns.to_list()]
    else:
        pnum.columns = [str(c) for c in pnum.columns.to_list()]
    num_wide_cols = [f"{feat}_w{wk}" for feat in numeric_cols for wk in weeks if f"{feat}_w{wk}" in pnum.columns]
    num_wide = pnum.reindex(columns=num_wide_cols)

    pcat = df_reset.pivot_table(index=id_cols, columns=seq_col, values=categorical_cols, aggfunc='first')
    if isinstance(pcat.columns, pd.MultiIndex):
        pcat.columns = [f"{feat}_w{int(wk) if str(wk).replace('.0','').isdigit() else wk}" for feat, wk in pcat.columns.to_list()]
    else:
        pcat.columns = [str(c) for c in pcat.columns.to_list()]
    cat_wide_cols = [f"{feat}_w{wk}" for feat in categorical_cols for wk in weeks if f"{feat}_w{wk}" in pcat.columns]
    cat_wide = pcat.reindex(columns=cat_wide_cols)

    return num_wide, cat_wide

In [17]:
# получаем wide-таблицы
num_wide, cat_wide = pivot_weeks(df_features.reset_index(drop=False), index_columns, sequential_columns[0],
                                 numeric_cols, categorical_cols, COUNT_WEEKS)
num_wide.shape, cat_wide.shape


((20852, 756), (20852, 348))

In [18]:
def make_cat_features_from_wide(cat_wide: pd.DataFrame, categorical_cols, COUNT_WEEKS: int, changes_as_bool: bool = False):
    """Для каждого categorical_cols возвращает first,last,mode,changes (DataFrame).
    Параметры:
      changes_as_bool — если True, колонка {feat}_changes будет булевой (True если было хотя бы одно изменение),
                        иначе целочисленная (количество смен).
    Возвращает: (cat_tab, cat_tab_columns, cat_wide_columns, num_feats)
    """
    weeks = list(range(1, COUNT_WEEKS))
    cat_feats = {}
    num_feats = []
    for feat in categorical_cols:
        cols = [f"{feat}_w{wk}" for wk in weeks if f"{feat}_w{wk}" in cat_wide.columns]

        if cols:
            # берём значения по неделям, заменяем NA на 'missing' и приводим к str
            C = cat_wide[cols].fillna("missing").astype(str)
            first_ser = C.get(f"{feat}_w1", C.iloc[:, 0]).astype(str)
            last_ser = C.get(f"{feat}_w{COUNT_WEEKS-1}", C.iloc[:, -1]).astype(str)
            # mode: безопасно получить первый модальный, иначе fallback на первый столбец
            try:
                mode_ser = C.mode(axis=1).iloc[:, 0].astype(str)
            except Exception:
                mode_ser = C.iloc[:, 0].astype(str)

            # changes: подсчёт смен между соседними неделями
            arr = C.to_numpy(dtype=object)
            if arr.shape[1] > 1:
                changes_cnt = (arr[:, 1:] != arr[:, :-1]).sum(axis=1).astype(np.int32)
            else:
                changes_cnt = np.zeros(C.shape[0], dtype=np.int32)

            if changes_as_bool:
                changes_ser = pd.Series(changes_cnt > 0, index=C.index, dtype=bool)
            else:
                changes_ser = pd.Series(changes_cnt, index=C.index, dtype=np.int32)

            cat_feats[f"{feat}_first"] = first_ser
            cat_feats[f"{feat}_last"] = last_ser
            cat_feats[f"{feat}_mode"] = mode_ser
            cat_feats[f"{feat}_changes"] = changes_ser
            num_feats.append(f"{feat}_changes")
        else:
            # нет wide-столбцов для признака — ставим заглушки
            idx = cat_wide.index
            cat_feats[f"{feat}_first"] = pd.Series("missing", index=idx, dtype=object)
            cat_feats[f"{feat}_last"] = pd.Series("missing", index=idx, dtype=object)
            cat_feats[f"{feat}_mode"] = pd.Series("missing", index=idx, dtype=object)
            if changes_as_bool:
                cat_feats[f"{feat}_changes"] = pd.Series(False, index=idx, dtype=bool)
            else:
                cat_feats[f"{feat}_changes"] = pd.Series(0, index=idx, dtype=np.int32)
            num_feats.append(f"{feat}_changes")

    cat_tab = pd.DataFrame(cat_feats, index=cat_wide.index)
    cat_tab_columns = [c for c in cat_tab.columns.tolist() if c not in num_feats]
    # Возвращаем: (таблица, список категориальных итоговых признаков, список колонок wide, список числовых _changes)
    return cat_tab, cat_tab_columns, cat_wide.columns.tolist(), num_feats


In [19]:
cat_tab, cat_tab_columns, cat_wide_columns, num_feats = make_cat_features_from_wide(cat_wide, categorical_cols, COUNT_WEEKS)
logger.info(f"Created categorical features: {cat_tab_columns}")
logger.info(f"Categorical wide columns: {cat_wide_columns}")
logger.info(f"Categorical features table shape: {cat_tab.shape}")
logger.info(f"Numerical features: {num_feats}")

TypeError: Invalid value 'missing' for dtype 'boolean'

In [ ]:
def make_num_features_from_wide(num_wide: pd.DataFrame, numeric_cols, COUNT_WEEKS: int):
    """Для каждого numeric_cols возвращает first,last,mean,std,trend,min,max,last_nonnull_count."""
    num_aggs = {}
    weeks = list(range(1, COUNT_WEEKS))
    for feat in numeric_cols:
        cols = [c for c in num_wide.columns if c.startswith(f"{feat}_w")]
        if len(cols) == 0:
            idx = num_wide.index
            num_aggs[f"{feat}_first"] = pd.Series(np.nan, index=idx)
            num_aggs[f"{feat}_last"]  = pd.Series(np.nan, index=idx)
            num_aggs[f"{feat}_mean"]  = pd.Series(np.nan, index=idx)
            num_aggs[f"{feat}_std"]   = pd.Series(np.nan, index=idx)
            num_aggs[f"{feat}_trend"] = pd.Series(0.0, index=idx)
            num_aggs[f"{feat}_min"]   = pd.Series(np.nan, index=idx)
            num_aggs[f"{feat}_max"]   = pd.Series(np.nan, index=idx)
            num_aggs[f"{feat}_last_nonnull_count"] = pd.Series(0, index=idx, dtype=int)
            continue

        mat = num_wide[cols].to_numpy(dtype=float)
        mask = ~np.isnan(mat)
        counts = mask.sum(axis=1)

        # first = week1 (most distant) fallback -> first available
        first_col = f"{feat}_w1"
        last_col  = f"{feat}_w{COUNT_WEEKS-1}"
        s_first = num_wide[first_col] if first_col in num_wide.columns else num_wide[cols].iloc[:, 0]
        s_last  = num_wide[last_col]  if last_col  in num_wide.columns else num_wide[cols].iloc[:, -1]

        num_aggs[f"{feat}_first"] = s_first.astype(float)
        num_aggs[f"{feat}_last"]  = s_last.astype(float)

        num_aggs[f"{feat}_mean"] = pd.Series(np.nanmean(mat, axis=1), index=num_wide.index)
        num_aggs[f"{feat}_std"]  = pd.Series(np.nanstd(mat, axis=1), index=num_wide.index)
        num_aggs[f"{feat}_min"]  = pd.Series(np.nanmin(mat, axis=1), index=num_wide.index)
        num_aggs[f"{feat}_max"]  = pd.Series(np.nanmax(mat, axis=1), index=num_wide.index)
        num_aggs[f"{feat}_last_nonnull_count"] = pd.Series(counts, index=num_wide.index)

        # trend: векторно (линейный slope) по реальным номерам недель в cols
        present_wks = np.array([int(c.rsplit('_w', 1)[1]) for c in cols], dtype=float)
        mean_y = np.nanmean(mat, axis=1)
        weighted_w = (mask * present_wks).sum(axis=1)
        mean_w = np.where(counts > 0, weighted_w / np.maximum(counts, 1), 0.0)
        num = ((present_wks - mean_w[:, None]) * (mat - mean_y[:, None])) * mask
        numerator = np.nansum(num, axis=1)
        denom = np.sum(((present_wks - mean_w[:, None]) ** 2) * mask, axis=1)
        slope = np.where(denom > 0, numerator / denom, 0.0)
        slope[counts < 2] = 0.0
        num_aggs[f"{feat}_trend"] = pd.Series(slope, index=num_wide.index)

    num_tab = pd.DataFrame(num_aggs, index=num_wide.index)
    return num_tab, num_tab.columns.tolist(), num_wide.columns.tolist()

In [ ]:
num_tab, num_tab_columns, num_wide_columns = make_num_features_from_wide(num_wide, numeric_cols, COUNT_WEEKS)
logger.info(f"Created numeric features: {num_tab_columns}")
logger.info(f"Numeric wide columns: {num_wide_columns}")
logger.info(f"Numeric features table shape: {num_tab.shape}")

/tmp/ipykernel_3308355/1963821778.py:32: RuntimeWarning: Mean of empty slice
  num_aggs[f"{feat}_mean"] = pd.Series(np.nanmean(mat, axis=1), index=num_wide.index)
/data/aturov/scoring/.venv/lib/python3.10/site-packages/numpy/lib/_nanfunctions_impl.py:2019: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/tmp/ipykernel_3308355/1963821778.py:34: RuntimeWarning: All-NaN slice encountered
  num_aggs[f"{feat}_min"]  = pd.Series(np.nanmin(mat, axis=1), index=num_wide.index)
/tmp/ipykernel_3308355/1963821778.py:35: RuntimeWarning: All-NaN slice encountered
  num_aggs[f"{feat}_max"]  = pd.Series(np.nanmax(mat, axis=1), index=num_wide.index)
/tmp/ipykernel_3308355/1963821778.py:40: RuntimeWarning: Mean of empty slice
  mean_y = np.nanmean(mat, axis=1)
/tmp/ipykernel_3308355/1963821778.py:46: RuntimeWarning: invalid value encountered in divide
  slope = np.where(denom > 0, numerator / denom, 0.0)
/tmp/ipykernel_3308355/1963821778.p

In [ ]:
df_table = tgt.join(num_wide[num_wide_columns], how='inner', on=index_columns)
logger.info(f"After joining numeric features, shape: {df_table.shape}")
df_table = df_table.join(cat_wide[cat_wide_columns], how='inner', on=index_columns)
logger.info(f"After joining wide categorical features, shape: {df_table.shape}")
df_table = df_table.join(num_tab[num_tab_columns], how='inner', on=index_columns)
logger.info(f"After joining numeric features, shape: {df_table.shape}")
df_table = df_table.join(cat_tab[cat_tab_columns], how='inner', on=index_columns)
logger.info(f"After joining categorical features, shape: {df_table.shape}")
df_table = df_table.join(cat_tab[num_feats], how='inner', on=index_columns)
logger.info(f"After joining categorical _changes features, shape: {df_table.shape}")


2025-10-30 15:56:55,669 | my_logger - INFO - After joining numeric features, shape: (14120, 253) | /tmp/ipykernel_3308355/1463284494.py:2
2025-10-30 15:56:55,983 | my_logger - INFO - After joining wide categorical features, shape: (14120, 369) | /tmp/ipykernel_3308355/1463284494.py:4
2025-10-30 15:56:56,337 | my_logger - INFO - After joining numeric features, shape: (14120, 873) | /tmp/ipykernel_3308355/1463284494.py:6


2025-10-30 15:56:56,739 | my_logger - INFO - After joining categorical features, shape: (14120, 960) | /tmp/ipykernel_3308355/1463284494.py:8
2025-10-30 15:56:57,098 | my_logger - INFO - After joining categorical _changes features, shape: (14120, 989) | /tmp/ipykernel_3308355/1463284494.py:10


In [ ]:
df_table.to_parquet(config.environment.data_processed_path / f'{NAME_DATAFRAME_TABLE}_{CURRENT_DATE}.parquet', index=True)
logger.info(f"Features table saved to {config.environment.data_processed_path / f'{NAME_DATAFRAME_TABLE}_{CURRENT_DATE}.parquet'}")
# Сохранение категориальных признаков и числовых в json
num_columns = num_wide_columns+num_tab_columns + num_feats
cat_columns = cat_wide_columns+ cat_tab_columns
with open(config.environment.data_processed_path / f'{NAME_DATAFRAME_TABLE}_num_columns_{CURRENT_DATE}.json', 'w') as f:
    json.dump(num_columns, f)
with open(config.environment.data_processed_path / f'{NAME_DATAFRAME_TABLE}_cat_columns_{CURRENT_DATE}.json', 'w') as f:
    json.dump(cat_columns, f)

2025-10-30 15:56:57,904 | my_logger - INFO - Features table saved to /data/aturov/scoring/data/processed/features_table_4_20_60_2025-10-30.parquet | /tmp/ipykernel_3308355/2801462665.py:2


### Добавим фичи из банковских данных

In [22]:
def age_full_years(df_part):
    # Робастный расчёт возраста (полные годы на дату открытия)
    df_part['date_open'] = pd.to_datetime(df_part['date_open'], errors='coerce')
    df_part['date_birth'] = pd.to_datetime(df_part['date_birth'], errors='coerce')

    # опционально: сохраняем дни для отладки/аналитики
    df_part['age_days'] = (df_part['date_open'] - df_part['date_birth']).dt.days

    # вычисляем полные годы (векторно, без apply)
    mask = df_part['date_open'].notna() & df_part['date_birth'].notna()
    y_open = df_part.loc[mask, 'date_open'].dt.year
    m_open = df_part.loc[mask, 'date_open'].dt.month
    d_open = df_part.loc[mask, 'date_open'].dt.day
    y_birth = df_part.loc[mask, 'date_birth'].dt.year
    m_birth = df_part.loc[mask, 'date_birth'].dt.month
    d_birth = df_part.loc[mask, 'date_birth'].dt.day

    age_years = y_open - y_birth - ((m_open < m_birth) | ((m_open == m_birth) & (d_open < d_birth))).astype(int)
    df_part.loc[mask, 'age'] = age_years

    # nullable integer и фильтрация не реалистичных значений
    df_part['age'] = df_part['age'].astype('Int64')
    df_part.loc[~df_part['age'].between(0, 120), 'age'] = pd.NA
    return df_part

In [23]:
def make_query(engine):
    """
    Функция для выполнения SQL-запроса к базе данных и получения данных в виде DataFrame.
    Параметры:
    - engine: SQLAlchemy engine для подключения к базе данных.
    Возвращает:
    - DataFrame с результатами запроса.
    """

    query = f"""
            SELECT 
                *
            FROM data_science.credits_subs_eldik_clean AS ce
        
    """

    data = pd.read_sql(query, engine)

    return data
df_part = make_query(clickhouse_engine)
logger.info(f"Data shape from ClickHouse: {df_part.shape}")
df_part.drop(columns=['phone_beeline', 
                      'inn_beeline', 'subs_eff_dt',  'interest_on_credit',
                      'match_phone', 'match_inn', 'overdue_max',
                        'total_overdue', 'status'], inplace=True)
df_part.date_birth = pd.to_datetime(df_part.date_birth, errors='coerce')
df_part.date_open = pd.to_datetime(df_part.date_open, errors='coerce')
df_part.dropna(subset=['date_open', 'date_birth'], inplace=True)

logger.info(f"Data shape from ClickHouse: {df_part.shape}")

logger.info(f"Data shape from ClickHouse: {df_part.shape}")
df_part = age_full_years(df_part)
df_part.drop(columns=['date_birth'], inplace=True)

cat_banking_features = ['sex',  'birthplace', 'marital_status', 
       #'name_region', 'city', 'street'
       ]
num_banking_features = ['age', 'age_days', 'prev_credit_count', 'sum_of_prev_credits',
                        'contract_length', #'interest_on_credit', 
                        'summa'
                        ]


for col in num_banking_features:
    df_part[col].fillna(-1, inplace=True)
for col in cat_banking_features:
    df_part[col] = df_part[col].astype(str)
    df_part[col].fillna('unknown', inplace=True)


df_part.drop_duplicates(subset=['id_request', 'inn_eldik','id_credit'], inplace=True)
logger.info(f"Data shape from ClickHouse after processing: {df_part.shape}")
df_part.set_index(index_columns, inplace=True)

2025-10-30 15:56:58,429 | my_logger - INFO - Data shape from ClickHouse: (18631, 23) | /tmp/ipykernel_3308355/4205655774.py:21
2025-10-30 15:56:58,450 | my_logger - INFO - Data shape from ClickHouse: (18631, 14) | /tmp/ipykernel_3308355/4205655774.py:30
2025-10-30 15:56:58,450 | my_logger - INFO - Data shape from ClickHouse: (18631, 14) | /tmp/ipykernel_3308355/4205655774.py:32
/tmp/ipykernel_3308355/4205655774.py:46: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_part[col].fillna(-1, inplace=True)
/tmp/ipykernel_3308355/420

In [24]:
df_part = df_part.join(df_table, how='inner', on=index_columns)
logger.info(f"Data shape after joining with features table: {df_part.shape}")

2025-10-30 15:56:59,063 | my_logger - INFO - Data shape after joining with features table: (13901, 998) | /tmp/ipykernel_3308355/2143672639.py:2


In [25]:
df_part.to_parquet(config.environment.data_processed_path / f'{NAME_DATAFRAME_BANKING}_{CURRENT_DATE}.parquet', index=True)
logger.info(f"Features saved to {config.environment.data_processed_path / f'{NAME_DATAFRAME_BANKING}_{CURRENT_DATE}.parquet'}")
cat_banking_features += cat_tab_columns + cat_wide_columns
num_banking_features += num_tab_columns + num_wide_columns
# Сохранение категориальных признаков и числовых в json
import json
num_columns = num_columns + num_banking_features
cat_columns = cat_columns + cat_banking_features
with open(config.environment.data_processed_path / f'{NAME_DATAFRAME_BANKING}_cat_columns_{CURRENT_DATE}.json', 'w') as f:
    json.dump(cat_columns, f)
with open(config.environment.data_processed_path / f'{NAME_DATAFRAME_BANKING}_num_columns_{CURRENT_DATE}.json', 'w') as f:
    json.dump(num_columns, f)

2025-10-30 15:56:59,864 | my_logger - INFO - Features saved to /data/aturov/scoring/data/processed/features_table_banking_4_20_60_2025-10-30.parquet | /tmp/ipykernel_3308355/2904767515.py:2
